In [ ]:
!pip install kaggle
!echo '{"username":"gr4ceg","key":"08da1952aa6a96f237cf1e4c73f05c26"}' > kaggle.json
!mkdir -p /root/.kaggle
!mv kaggle.json /root/.kaggle
!chmod 600 /root/.kaggle/kaggle.json
!kaggle competitions download -c cs-480-2024-spring
!unzip -q *.zip

In [3]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sns

from graphviz import Digraph

import torch
from torchvision import transforms
from PIL import Image

# Example function to extract features from images using a pre-trained EfficientNet model
from torchvision.models import efficientnet_b7

import numpy as np
from tqdm import tqdm  # Import tqdm for the progress bar

from torch.utils.data import DataLoader, Dataset

from catboost import CatBoostRegressor, Pool

In [4]:
# Load the data
train_data = pd.read_csv('/kaggle/working/data/train.csv')

ancillary_columns = train_data.columns[1:164]
print(ancillary_columns)

label_columns = train_data.columns[164:]
print(label_columns)

Index(['WORLDCLIM_BIO1_annual_mean_temperature',
       'WORLDCLIM_BIO12_annual_precipitation',
       'WORLDCLIM_BIO13.BIO14_delta_precipitation_of_wettest_and_dryest_month',
       'WORLDCLIM_BIO15_precipitation_seasonality',
       'WORLDCLIM_BIO4_temperature_seasonality',
       'WORLDCLIM_BIO7_temperature_annual_range',
       'SOIL_bdod_0.5cm_mean_0.01_deg', 'SOIL_bdod_100.200cm_mean_0.01_deg',
       'SOIL_bdod_15.30cm_mean_0.01_deg', 'SOIL_bdod_30.60cm_mean_0.01_deg',
       ...
       'VOD_X_1997_2018_multiyear_mean_m03',
       'VOD_X_1997_2018_multiyear_mean_m04',
       'VOD_X_1997_2018_multiyear_mean_m05',
       'VOD_X_1997_2018_multiyear_mean_m06',
       'VOD_X_1997_2018_multiyear_mean_m07',
       'VOD_X_1997_2018_multiyear_mean_m08',
       'VOD_X_1997_2018_multiyear_mean_m09',
       'VOD_X_1997_2018_multiyear_mean_m10',
       'VOD_X_1997_2018_multiyear_mean_m11',
       'VOD_X_1997_2018_multiyear_mean_m12'],
      dtype='object', length=163)
Index(['X4_mean', 'X11_

In [5]:
# Split features and targets
image_paths = 'data/train_images/' + train_data.iloc[:, 0].astype(str) + '.jpeg'

ancillary_data = train_data.iloc[:, 1:164]
labels = train_data.iloc[:, 164:]

# Split the data into training and validation sets
train_ancillary, val_ancillary, train_labels, val_labels, train_images, val_images = train_test_split(
    ancillary_data, labels, image_paths, test_size=0.20, random_state=42)

In [6]:
train_images = train_images.to_numpy()
val_images = val_images.to_numpy()

In [7]:
print(f"Training ancillary data: {train_ancillary.shape}")
print(f"Validation ancillary data: {val_ancillary.shape}")
print(f"Training labels: {train_labels.shape}")
print(f"Validation labels: {val_labels.shape}")
print(f"Training images: {len(train_images)}")
print(f"Validation images: {len(val_images)}")

Training ancillary data: (34690, 163)
Validation ancillary data: (8673, 163)
Training labels: (34690, 6)
Validation labels: (8673, 6)
Training images: 34690
Validation images: 8673


In [8]:
print(image_paths.head())
print(ancillary_data[:5])
print(labels[:5])

0    data/train_images/101801795.jpeg
1    data/train_images/115813315.jpeg
2    data/train_images/173551949.jpeg
3    data/train_images/148811120.jpeg
4    data/train_images/195108876.jpeg
Name: id, dtype: object
   WORLDCLIM_BIO1_annual_mean_temperature  \
0                               21.478968   
1                               26.927639   
2                               27.336945   
3                               25.558649   
4                               25.204723   

   WORLDCLIM_BIO12_annual_precipitation  \
0                            772.404785   
1                           1456.733276   
2                            992.366638   
3                           2246.017822   
4                           2309.776123   

   WORLDCLIM_BIO13.BIO14_delta_precipitation_of_wettest_and_dryest_month  \
0                                         110.047623                       
1                                         329.366669                       
2                           

In [ ]:
# Plot histograms for each trait
traits = labels.columns
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(15, 10))
axes = axes.flatten()

for i, trait in enumerate(traits):
    sns.histplot(labels[trait], kde=True, ax=axes[i])
    axes[i].set_title(f'Distribution of {trait}')
    axes[i].set_xlabel(trait)

plt.tight_layout()
plt.show()

In [ ]:
corr = pd.DataFrame(ancillary_scaled, columns=ancillary_data.columns).corr()
plt.figure(figsize=(12, 10))
sns.heatmap(corr, cmap='coolwarm', annot=False, fmt='.2f')
plt.title('Correlation Heatmap of Ancillary Features')
plt.show()

In [ ]:
dot = Digraph()

dot.node('A', 'Input: Image Features (CNN)')
dot.node('B', 'Input: Ancillary Features')
dot.node('C', 'Concatenate Features')
dot.node('D', 'CatBoost Regressor')
dot.node('E', 'Trait Predictions')

dot.edges(['AC', 'BC', 'CD', 'DE'])

# Save the diagram to a file
dot.render('model_architecture', format='png')

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [10]:
# Define the augmentation pipeline
augmentation_pipeline = transforms.Compose([
#     transforms.RandomHorizontalFlip(),
#     transforms.RandomVerticalFlip(),
#     transforms.ColorJitter(
#         brightness=0.1,
#         contrast=0.1,
#         saturation=0.1
#     ),
    transforms.ToTensor(),
#     transforms.Lambda(lambda x: x.clamp(0, 1))
])

# Example function to load and preprocess an image
def load_and_preprocess_image(image_path):
    image = Image.open(image_path).convert('RGB')
    image = augmentation_pipeline(image)
    return image

In [13]:
# Load pre-trained EfficientNet B7 model and move it to GPU
effnet_model = efficientnet_b7(pretrained=True)

# try using the last layer withour rthe pooling
# effnet_model = torch.nn.Sequential(*list(effnet_model.children())[:-2], torch.nn.AdaptiveAvgPool2d((1, 1)))
effnet_model = torch.nn.Sequential(*list(effnet_model.children())[:-1])
effnet_model = effnet_model.to(device)  # Move the model to GPU

In [14]:
class ImageDataset(Dataset):
    def __init__(self, image_paths):
        self.image_paths = image_paths

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = load_and_preprocess_image(self.image_paths[idx])
        return image

def extract_features_dataloader(dataloader):
    features = []
    effnet_model.eval()
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Extracting Features with DataLoader"):
            batch = batch.to(device)
            batch_features = effnet_model(batch).flatten(start_dim=1)
            features.append(batch_features.cpu().numpy())
    return torch.tensor(np.concatenate(features, axis=0))

In [15]:
train_dataset = ImageDataset(train_images)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=False)
train_image_features = extract_features_dataloader(train_dataloader)

val_dataset = ImageDataset(val_images)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)
val_image_features = extract_features_dataloader(val_dataloader)

Extracting Features with DataLoader: 100%|██████████| 272/272 [00:27<00:00, 10.07it/s]


In [ ]:
# Assuming train_ancillary, train_image_features, and train_labels are already defined
X_train_combined = np.concatenate([train_ancillary, train_image_features], axis=1)
X_val_combined = np.concatenate([val_ancillary, val_image_features], axis=1)

# Split data into training and validation sets
# X_train, X_val, y_train, y_val = train_test_split(X_train_combined, train_labels, test_size=0.2, random_state=42)

# Dictionary to store models and their training logs
models = {}
training_logs = {}

# Train separate CatBoost models for each trait with GPU, early stopping, and multi-threading
for i, trait in tqdm(enumerate(labels.columns), total=len(labels.columns), desc="Training Models"):
    model = CatBoostRegressor(
        iterations=500,  # Reduced iterations
        depth=6,
        learning_rate=0.1,
        loss_function='RMSE',
        task_type='GPU',  # Enable GPU training
        early_stopping_rounds=50,  # Enable early stopping
        thread_count=-1,  # Use all available CPU cores
        logging_level='Verbose'  # Enable detailed logging
    )
    
    # Create Pool for validation set
#     train_pool = Pool(X_train, y_train.iloc[:, i])
#     val_pool = Pool(X_val, y_val.iloc[:, i])
    train_pool = Pool(X_train_combined, train_labels.iloc[:, i])
    val_pool = Pool(X_val_combined, val_labels.iloc[:, i])
    
    # Fit the model
    model.fit(train_pool, eval_set=val_pool, verbose=100)  # Adjust verbose as needed

    # Store the model
    models[trait] = model

    # Store the training logs
    training_logs[trait] = model.get_evals_result()

Training Models:   0%|          | 0/6 [00:00<?, ?it/s]

0:	learn: 0.1353396	test: 0.1363007	best: 0.1363007 (0)	total: 4.75s	remaining: 39m 30s
100:	learn: 0.1186654	test: 0.1240133	best: 0.1240133 (100)	total: 21.7s	remaining: 1m 25s
200:	learn: 0.1153299	test: 0.1235116	best: 0.1235116 (200)	total: 38s	remaining: 56.6s
300:	learn: 0.1125424	test: 0.1232373	best: 0.1232262 (296)	total: 54.3s	remaining: 35.9s
400:	learn: 0.1101331	test: 0.1230300	best: 0.1230207 (386)	total: 1m 10s	remaining: 17.4s


Training Models:  17%|█▋        | 1/6 [02:27<12:17, 147.48s/it]

499:	learn: 0.1079751	test: 0.1229340	best: 0.1229325 (494)	total: 1m 26s	remaining: 0us
bestTest = 0.1229325277
bestIteration = 494
Shrink model to first 495 iterations.
0:	learn: 6.8150829	test: 6.8454593	best: 6.8454593 (0)	total: 186ms	remaining: 1m 32s
100:	learn: 5.9354541	test: 6.1382688	best: 6.1382688 (100)	total: 17.1s	remaining: 1m 7s
200:	learn: 5.7484638	test: 6.0997609	best: 6.0997609 (200)	total: 33.7s	remaining: 50.1s


In [ ]:
# prev_logs = logs
# print(prev_logs)

In [ ]:
# Plotting the training logs
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(18, 10))
axes = axes.flatten()

# Create a subplot for each trait
for i, (trait, logs) in enumerate(training_logs.items()):
    ax = axes[i]
    
    # Plot Train and Validation RMSE on the primary y-axis
    ax.plot(logs['learn']['RMSE'], label='Train RMSE', color='blue')
    ax.plot(logs['validation']['RMSE'], label='Validation RMSE', color='orange')
    
    ax.set_title(f'CatBoost Training for {trait}')
    ax.set_xlabel('Iterations')
    ax.set_ylabel('RMSE')
    
    # Create a secondary y-axis to plot overfitting
    ax2 = ax.twinx()
    overfitting = np.abs(np.array(logs['learn']['RMSE'][:220]) - np.array(logs['validation']['RMSE'][:220]))
    ax.plot(overfitting, label='Overfitting |Train - Validation|, LR=0.01', color='red', linestyle='--')
    ax.set_ylabel('Overfitting (RMSE Difference)')
    
    # Combine legends from both axes
    lines, labels = ax.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax2.legend(lines + lines2, labels + labels2, loc='upper right')


plt.tight_layout()
plt.savefig('catboost.png')
plt.show()



In [ ]:
from sklearn.metrics import r2_score

# X_val_combined = np.concatenate([val_ancillary, val_image_features], axis=1)

r2_scores = {}
for i, trait in tqdm(enumerate(val_labels.columns), total=len(val_labels.columns), desc="Validating Models"):
    y_pred = models[trait].predict(X_val_combined)  # Predictions use GPU if CatBoost model was trained on GPU
    r2_scores[trait] = r2_score(val_labels.iloc[:, i], y_pred)

print("R2 Scores:", r2_scores)

In [ ]:
# Load the data
test_data = pd.read_csv('/kaggle/working/data/test.csv')

ancillary_columns = test_data.columns[1:164]
print(ancillary_columns)

In [ ]:
# Split features and targets
test_image_paths = 'data/test_images/' + test_data.iloc[:, 0].astype(str) + '.jpeg'

test_ancillary_data = test_data.iloc[:, 1:164]
test_labels = test_data.iloc[:, 164:]

test_image_paths = test_image_paths.to_numpy()

In [ ]:
# Generate predictions for the test set
test_dataset = ImageDataset(test_image_paths)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

test_image_features = extract_features_dataloader(test_dataloader)  # Define test_image_paths

X_test_combined = np.concatenate([test_ancillary_data, test_image_features], axis=1)

test_ids = test_data['id'].values  # Extract test IDs from the test dataset

submission = pd.DataFrame({'id': test_ids})  # Define test_ids
for trait in models:
    submission[trait] = models[trait].predict(X_test_combined)

submission.to_csv('submission.csv', index=False)